In [ ]:
!pip install exchange_calendars yfinance stockstats
!apt install chromium-driver
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 189 kB 7.9 MB/s 
     |████████████████████████████████| 62 kB 798 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'chromium-chromedriver' instead of 'chromium-driver'
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-code

In [ ]:
%cd drive/MyDrive/finance
%load_ext autoreload

/content/drive/MyDrive/finance


In [ ]:
%autoreload 2
import exchange_calendars as tc
import numpy as np
import pandas as pd
import yfinance as yf
from stockstats import StockDataFrame as Sdf
import concurrent.futures
import datetime
import os
from collections import defaultdict
from bs4 import BeautifulSoup
from selenium import webdriver
from finance.trader.data.yahoodownloader import YahooDownloader
from finance.config import INDICATORS, TRAIN_START, TRAIN_END, VAL_END, VAL_START, TEST_END, TEST_START
from finance.config_tickers import SENTIMENT_TICKERS

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 30) 
pd.set_option('display.max_rows', 100)

In [ ]:
start = TRAIN_START
end = TEST_END
ticker_list = SENTIMENT_TICKERS
indicators = INDICATORS
yd = YahooDownloader()
data = yd.download(ticker_list=ticker_list, start_date=start, end_date=end, interval='1D')
data = yd.clean_data(data)
#data = yd.add_indicators(data, indicators)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (19917, 9)
Clean data for CL=F
Data clean for CL=F is finished.
Clean data for GC=F
Data clean for GC=F is finished.
Clean data for ZB=F
Data clean for ZB=F is finished.
Clean data for ZF=F
Data clean for ZF=F is finished.
Clean data for ZN=F
Data clean for ZN=F is finished.
Clean data fo

In [ ]:
data.isnull().sum()

time       0
open      33
high      33
low       33
close     33
adjcp     33
volume    33
tic        0
dtype: int64

In [ ]:
data

,time,open,high,low,close,adjcp,volume,tic
0,2015-01-02,53.76,55.11,52.03,52.69,52.69,"268,708.00",CL=F
1,2015-01-05,52.61,52.73,49.68,50.04,50.04,"375,782.00",CL=F
2,2015-01-06,50.00,50.37,47.55,47.93,47.93,"451,642.00",CL=F
3,2015-01-07,48.00,49.31,46.83,48.65,48.65,"460,083.00",CL=F
4,2015-01-08,48.78,49.65,47.73,48.79,48.79,"362,081.00",CL=F
...,...,...,...,...,...,...,...,...
19945,2022-11-25,3.76,3.78,3.74,3.75,3.75,0.00,^TYX
19946,2022-11-28,3.72,3.77,3.70,3.75,3.75,0.00,^TYX
19947,2022-11-29,3.75,3.81,3.73,3.80,3.80,0.00,^TYX
19948,2022-11-30,3.79,3.85,3.78,3.82,3.82,0.00,^TYX


In [ ]:
pd.set_option('display.max_columns', 30) 
pd.set_option('display.max_rows', 100)
data.reset_index(drop=True).head(100)

,time,open,high,low,close,adjcp,volume,tic,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,wr_30,atr_30,supertrend,mfi_30,close_30_tema
0,2015-01-02,53.76,55.11,52.03,52.69,52.69,"268,708.00",CL=F,0.00,NaN,NaN,NaN,NaN,NaN,52.69,52.69,-78.57,3.08,62.81,0.50,52.69
1,2015-01-02,"1,184.00","1,194.50","1,169.50","1,186.00","1,186.00",138.00,GC=F,0.00,NaN,NaN,NaN,NaN,NaN,"1,186.00","1,186.00",-34.00,25.00,"1,257.00",0.50,"1,186.00"
2,2015-01-02,144.34,145.94,143.97,145.50,145.50,"256,841.00",ZB=F,0.00,NaN,NaN,NaN,NaN,NaN,145.50,145.50,-22.22,1.97,150.86,0.50,145.50
3,2015-01-02,118.81,119.19,118.67,119.09,119.09,"479,043.00",ZF=F,0.00,NaN,NaN,NaN,NaN,NaN,119.09,119.09,-18.18,0.52,120.48,0.50,119.09
4,2015-01-02,126.61,127.33,126.41,127.16,127.16,"768,078.00",ZN=F,0.00,NaN,NaN,NaN,NaN,NaN,127.16,127.16,-18.64,0.92,129.63,0.50,127.16
5,2015-01-02,109.27,109.34,109.23,109.32,109.32,"155,339.00",ZT=F,0.00,NaN,NaN,NaN,NaN,NaN,109.32,109.32,-20.00,0.12,109.64,0.50,109.32
6,2015-01-02,1.69,1.71,1.60,1.62,1.62,0.00,^FVX,0.00,NaN,NaN,NaN,NaN,NaN,1.62,1.62,-82.86,0.11,1.97,0.50,1.62
7,2015-01-02,0.03,0.03,0.01,0.01,0.01,0.00,^IRX,0.00,NaN,NaN,NaN,NaN,NaN,0.01,0.01,-100.00,0.01,0.07,0.50,0.01
8,2015-01-02,2.20,2.21,2.10,2.12,2.12,0.00,^TNX,0.00,NaN,NaN,NaN,NaN,NaN,2.12,2.12,-81.08,0.11,2.49,0.50,2.12
9,2015-01-02,2.77,2.78,2.68,2.70,2.70,0.00,^TYX,0.00,NaN,NaN,NaN,NaN,NaN,2.70,2.70,-80.95,0.11,3.04,0.50,2.70


In [ ]:
data[data['cci_30'].isnull()]

,time,open,high,low,close,adjcp,volume,tic,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,wr_30,atr_30,supertrend,mfi_30,close_30_tema
0,2015-01-02,53.76,55.11,52.03,52.69,52.69,"268,708.00",CL=F,0.00,NaN,NaN,NaN,NaN,NaN,52.69,52.69,-78.57,3.08,62.81,0.50,52.69
1,2015-01-02,"1,184.00","1,194.50","1,169.50","1,186.00","1,186.00",138.00,GC=F,0.00,NaN,NaN,NaN,NaN,NaN,"1,186.00","1,186.00",-34.00,25.00,"1,257.00",0.50,"1,186.00"
2,2015-01-02,144.34,145.94,143.97,145.50,145.50,"256,841.00",ZB=F,0.00,NaN,NaN,NaN,NaN,NaN,145.50,145.50,-22.22,1.97,150.86,0.50,145.50
3,2015-01-02,118.81,119.19,118.67,119.09,119.09,"479,043.00",ZF=F,0.00,NaN,NaN,NaN,NaN,NaN,119.09,119.09,-18.18,0.52,120.48,0.50,119.09
4,2015-01-02,126.61,127.33,126.41,127.16,127.16,"768,078.00",ZN=F,0.00,NaN,NaN,NaN,NaN,NaN,127.16,127.16,-18.64,0.92,129.63,0.50,127.16
5,2015-01-02,109.27,109.34,109.23,109.32,109.32,"155,339.00",ZT=F,0.00,NaN,NaN,NaN,NaN,NaN,109.32,109.32,-20.00,0.12,109.64,0.50,109.32
6,2015-01-02,1.69,1.71,1.60,1.62,1.62,0.00,^FVX,0.00,NaN,NaN,NaN,NaN,NaN,1.62,1.62,-82.86,0.11,1.97,0.50,1.62
7,2015-01-02,0.03,0.03,0.01,0.01,0.01,0.00,^IRX,0.00,NaN,NaN,NaN,NaN,NaN,0.01,0.01,-100.00,0.01,0.07,0.50,0.01
8,2015-01-02,2.20,2.21,2.10,2.12,2.12,0.00,^TNX,0.00,NaN,NaN,NaN,NaN,NaN,2.12,2.12,-81.08,0.11,2.49,0.50,2.12
9,2015-01-02,2.77,2.78,2.68,2.70,2.70,0.00,^TYX,0.00,NaN,NaN,NaN,NaN,NaN,2.70,2.70,-80.95,0.11,3.04,0.50,2.70
